In [1]:
import os

import sqlite3 as sq
from sqlalchemy import create_engine, inspect, Table, MetaData
from sqlalchemy.orm import sessionmaker

import pandas as pd


In [2]:
mta_data_engine = create_engine("sqlite:///mta_data.db")


In [3]:
def insert_table_from_csv(csv_list, engine):
    for file in csv_list:
        with open(file, 'r') as f:
            data = pd.read_csv(f)
            data.columns = data.columns.str.strip()
        data.to_sql(os.path.splitext(file)[0], con=mta_data_engine, index=False, if_exists='replace')
    

In [4]:
insert_table_from_csv(['mta_stations_loc.csv', 'mta_complex_id.csv'], mta_data_engine)


In [5]:
insp = inspect(mta_data_engine)
print(insp.get_table_names())


['mta_complex_id', 'mta_data', 'mta_stations_loc']


In [30]:
temp_df = pd.read_sql('''
            SELECT a.booth, a.unit, a.scp, a.station, a.linename, a.division, 
              MAX(a.entries)-MIN(a.entries) AS net_entries,
              MAX(a.exits)-MIN(a.exits) AS net_exits,
              CAST(b.complex_id AS int) AS complex_id, 
              c."GTFS Latitude" AS lat, c."GTFS Longitude" AS lon
            FROM mta_data a
            JOIN mta_complex_id b
            JOIN mta_stations_loc c
            ON a.booth = b.booth AND a.unit = b.remote AND b.complex_ID = c."Complex ID"
            WHERE a.date >= '06/01/2017' 
              AND a.date <= '08/31/2017' 
              AND a.desc = 'REGULAR'
            GROUP BY a.booth, a.unit, a.scp;
            ''', 
            mta_data_engine)


In [31]:
mta_df = pd.read_sql('''
            SELECT booth, unit, station, linename, division, SUM(net_entries)+SUM(net_exits) AS net_rides, complex_id, lat, lon
            FROM
            (SELECT a.booth, a.unit, a.scp, a.station, a.linename, a.division, 
              MAX(a.entries)-MIN(a.entries) AS net_entries,
              MAX(a.exits)-MIN(a.exits) AS net_exits,
              CAST(b.complex_id AS int) AS complex_id, 
              c."GTFS Latitude" AS lat, c."GTFS Longitude" AS lon
            FROM mta_data a
            JOIN mta_complex_id b
            JOIN mta_stations_loc c
            ON a.booth = b.booth AND a.unit = b.remote AND b.complex_ID = c."Complex ID"
            WHERE a.date >= '06/01/2017' 
              AND a.date <= '08/31/2017' 
              AND a.desc = 'REGULAR'
            GROUP BY a.booth, a.unit, a.scp)
            GROUP BY complex_id;
            ''', 
            mta_data_engine)

mta_df

,booth,unit,station,linename,division,net_rides,complex_id,lat,lon
0,R203,R043,WALL ST,45,IRT,7652738465,413,40.707557,-74.011862
1,A002,R051,59 ST,NQR456W,BMT,3845810339,613,40.762526,-73.967967
2,R304,R206,125 ST,23,IRT,3841903292,439,40.807754,-73.945495
3,A022,R022,34 ST-HERALD SQ,BDFMNQRW,BMT,3321724474,607,40.749567,-73.987950
4,R249,R179,86 ST,456,IRT,2947593228,397,40.779492,-73.955589
...,...,...,...,...,...,...,...,...,...
418,J037,R009,121 ST,JZ,BMT,84209,80,40.700492,-73.828294
419,N186,R418,BEACH 105 ST,AS,IND,77874,202,40.583209,-73.827559
420,S102,R165,TOMPKINSVILLE,1,SRT,64990,502,40.636949,-74.074835
421,N183,R415,BROAD CHANNEL,AS,IND,31961,199,40.608382,-73.815925
